In [29]:
from PIL import Image
import numpy as np

def overlay_mask_with_white(original_path: str, mask_path: str, out_path: str = "overlaid_result.png", white_threshold: int = 200):
    """
    Overlap mask onto original image so that mask's white regions become white on the original.
    Works with common image formats (BMP, TIFF, PNG, etc.).
    
    Args:
        original_path: path to the original image (color or grayscale).
        mask_path: path to the mask image (grayscale or color). White areas in mask will be applied.
        out_path: path to save the resulting image.
        white_threshold: grayscale threshold (0-255). Mask pixels >= threshold are considered white.
    
    Returns:
        out_path (str) - path to the saved result image.
    """
    orig = Image.open(original_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")  # convert mask to grayscale intensity

    # Resize mask to original if needed, using NEAREST to preserve binary regions
    if mask.size != orig.size:
        mask = mask.resize(orig.size, resample=Image.NEAREST)
    
    orig_arr = np.array(orig)
    mask_arr = np.array(mask)
    
    # boolean mask where mask is white or near white
    white_mask = mask_arr >= white_threshold
    
    # expand to 3 channels
    white_mask_3 = np.stack([white_mask]*3, axis=-1)
    
    # apply: set masked pixels to white (255,255,255)
    result = orig_arr.copy()
    result[white_mask_3] = 255
    
    res_img = Image.fromarray(result)
    res_img.save(out_path)
    return out_path


In [30]:
# ----------------- MAIN -----------------
root = "iris_db/IITD_v1"
segmented_root = "overlay"

orig_dir = os.path.join(root, "orig")
mask_dir = os.path.join(root, "mask_irisseg")

# Traverse subfolders in sorted order
for subfolder in sorted(os.listdir(orig_dir)):
    orig_sub = os.path.join(orig_dir, subfolder)
    mask_sub = os.path.join(mask_dir, subfolder)
    result_sub = os.path.join(segmented_root, subfolder)

    if not os.path.isdir(orig_sub):
        continue

    # Sorted files in each subfolder
    for filename in sorted(os.listdir(orig_sub)):
        if filename.lower().endswith(".bmp"):
            img_path = os.path.join(orig_sub, filename)

            # assume same stem but mask in .tiff
            stem = os.path.splitext(filename)[0]
            mask_path = os.path.join(mask_sub, stem + ".tiff")

            # force .png for saving
            save_path = os.path.join(result_sub, stem + ".bmp")

            if os.path.exists(mask_path):
                apply_mask(img_path, mask_path, save_path)
                print(f"Saved: {save_path}")
            else:
                print(f"Mask not found: {mask_path}")
            break
    break


Saved: overlay/100_L/100_L_1.bmp
